In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




> Evaluation metrics - PSNR/SSIM


In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from skimage import io
from skimage.metrics import structural_similarity as ssim
import numpy as np


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    ssim_scores = []
    psnr_scores = []

    for batch_idx, (japanese_images, english_images) in enumerate(dataloader):
        # Perform inference using the generator
        with torch.no_grad():
            generated_images = model(japanese_images)

        # Convert generated images to NumPy arrays
        generated_images = generated_images.cpu().numpy()
        english_images = english_images.numpy()

        # Compute SSIM and PSNR for each image in the batch
        for i in range(batch_size):
            generated_image = np.transpose(generated_images[i], (1, 2, 0))
            target_image = np.transpose(english_images[i], (1, 2, 0))

            ssim_score = ssim(generated_image, target_image, multichannel=True)
            psnr_score = -10 * np.log10(np.mean((generated_image - target_image) ** 2))

            ssim_scores.append(ssim_score)
            psnr_scores.append(psnr_score)

    # Calculate average SSIM and PSNR scores
    avg_ssim = np.mean(ssim_scores)
    avg_psnr = np.mean(psnr_scores)

    return avg_ssim, avg_psnr

# Evaluate the model
avg_ssim, avg_psnr = evaluate(generator, eval_loader)

print(f'Average SSIM: {avg_ssim}')
print(f'Average PSNR: {avg_psnr}')